In [77]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import SVD

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split

import pandas as pd
import numpy as np

1.Датасет ml-latest
2.Вспомнить подходы, которые мы разбирали
3.Выбрать понравившийся подход к гибридным системам
4.Написать свою

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies_with_tags = pd.merge(movies, tags, how = 'left', on = 'movieId')

In [4]:
movies_with_tags_with_ratings = pd.merge(movies_with_tags, ratings, how = 'inner', left_on = ['userId', 'movieId'], right_on =  ['userId', 'movieId'])

In [5]:
movies_with_tags_with_ratings

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09,4.0,978575760
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09,3.5,1525286001
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09,4.0,1528843890
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09,4.0,1528843890
...,...,...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62.0,star wars,1.528935e+09,4.0,1528934550
3472,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,anime,1.537099e+09,3.5,1537098554
3473,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,comedy,1.537099e+09,3.5,1537098554
3474,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184.0,gintama,1.537099e+09,3.5,1537098554


In [16]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [17]:
movies_with_tags_with_ratings['genres'] = movies_with_tags_with_ratings['genres'].apply(change_string)

In [18]:
movies_with_tags_with_ratings['feature'] = movies_with_tags_with_ratings['genres'] + ' ' + movies_with_tags_with_ratings['tag']

In [68]:
movies_with_tags_with_ratings

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y,feature
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,336.0,pixar,1.139046e+09,4.0,1122227329,Adventure Animation Children Comedy Fantasy pixar
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,474.0,pixar,1.137207e+09,4.0,978575760,Adventure Animation Children Comedy Fantasy pixar
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,567.0,fun,1.525286e+09,3.5,1525286001,Adventure Animation Children Comedy Fantasy fun
3,2,Jumanji (1995),Adventure Children Fantasy,62.0,fantasy,1.528844e+09,4.0,1528843890,Adventure Children Fantasy fantasy
4,2,Jumanji (1995),Adventure Children Fantasy,62.0,magic board game,1.528844e+09,4.0,1528843890,Adventure Children Fantasy magic board game
...,...,...,...,...,...,...,...,...,...
3471,187595,Solo: A Star Wars Story (2018),Action Adventure Children SciFi,62.0,star wars,1.528935e+09,4.0,1528934550,Action Adventure Children SciFi star wars
3472,193565,Gintama: The Movie (2010),Action Animation Comedy SciFi,184.0,anime,1.537099e+09,3.5,1537098554,Action Animation Comedy SciFi anime
3473,193565,Gintama: The Movie (2010),Action Animation Comedy SciFi,184.0,comedy,1.537099e+09,3.5,1537098554,Action Animation Comedy SciFi comedy
3474,193565,Gintama: The Movie (2010),Action Animation Comedy SciFi,184.0,gintama,1.537099e+09,3.5,1537098554,Action Animation Comedy SciFi gintama


In [69]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movies_with_tags_with_ratings['feature'].values)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [98]:
test = change_string("Crime Drama Mystery Thriller thought-provoking")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [99]:
res

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1.]]),
 array([[ 520, 3056, 2191,  661, 2285, 1438, 2192, 2749, 2193, 2194,  320,
         3374,  368,  706, 2745, 1453, 3289, 1604, 3077,  900]],
       dtype=int64))

In [106]:
movies_with_tags_with_ratings.iloc[res[1][0]].sort_values(by = 'rating')[-10:].title.values

array(['Dark Knight, The (2008)', 'Silence (2016)',
       '13 Going on 30 (2004)', 'Funny Games U.S. (2007)',
       'Top Hat (1935)', 'Dark Knight Rises, The (2012)',
       'Pulp Fiction (1994)', 'Pulp Fiction (1994)',
       'Talented Mr. Ripley, The (1999)', 'Primal Fear (1996)'],
      dtype=object)

In [6]:
dataset = pd.DataFrame({
    'uid': movies_with_tags_with_ratings.userId,
    'iid': movies_with_tags_with_ratings.title,
    'rating': movies_with_tags_with_ratings.rating
})

In [8]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
trainset, testset = train_test_split(data, test_size=0.15)

In [9]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10, 15, 20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.4, 0.6], 'n_factors' : [10 ,20] }
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=10)

gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.6648533964480189
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 20}


In [10]:
algo = gs.best_estimator['rmse']
algo.fit(trainset)
predictions = algo.test(testset)

In [61]:
predictions

[Prediction(uid=474.0, iid='Hearts and Minds (1996)', r_ui=4.0, est=3.7959738319007803, details={'was_impossible': False}),
 Prediction(uid=474.0, iid='Stage Fright (1950)', r_ui=3.5, est=3.778100254680227, details={'was_impossible': False}),
 Prediction(uid=125.0, iid='Lady Vengeance (Sympathy for Lady Vengeance) (Chinjeolhan geumjassi) (2005)', r_ui=5.0, est=4.551898963914701, details={'was_impossible': False}),
 Prediction(uid=474.0, iid='Control Room (2004)', r_ui=4.0, est=3.778100254680227, details={'was_impossible': False}),
 Prediction(uid=474.0, iid='Jane Eyre (1970)', r_ui=3.5, est=3.778100254680227, details={'was_impossible': False}),
 Prediction(uid=477.0, iid='Rear Window (1954)', r_ui=3.5, est=4.0669777744837186, details={'was_impossible': False}),
 Prediction(uid=599.0, iid='Pulp Fiction (1994)', r_ui=5.0, est=4.845364573629731, details={'was_impossible': False}),
 Prediction(uid=319.0, iid='Lion King, The (1994)', r_ui=5.0, est=4.449166654736825, details={'was_impossible

In [66]:
title_genres = {}

for i, j in movies_with_tags_with_ratings.iterrows():
    title_genres[j.title] = j.feature

In [67]:
title_genres

{'Toy Story (1995)': 'Adventure Animation Children Comedy Fantasy fun',
 'Jumanji (1995)': 'Adventure Children Fantasy game',
 'Grumpier Old Men (1995)': 'Comedy Romance old',
 'Father of the Bride Part II (1995)': 'Comedy remake',
 'Sabrina (1995)': 'Comedy Romance remake',
 'American President, The (1995)': 'Comedy Drama Romance president',
 'Nixon (1995)': 'Drama president',
 'Casino (1995)': 'Crime Drama Mafia',
 'Sense and Sensibility (1995)': 'Drama Romance Jane Austen',
 'Get Shorty (1995)': 'Comedy Crime Thriller Hollywood',
 'Copycat (1995)': 'Crime Drama Horror Mystery Thriller serial killer',
 'Leaving Las Vegas (1995)': 'Drama Romance alcoholism',
 'Othello (1995)': 'Drama Shakespeare',
 'Persuasion (1995)': 'Drama Romance Jane Austen',
 'City of Lost Children, The (Cité des enfants perdus, La) (1995)': 'Adventure Drama Fantasy Mystery SciFi kidnapping',
 'Dangerous Minds (1995)': 'Drama teacher',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)': 'Mystery SciFi Thriller twist e

In [119]:
def recommend_for_user(user_id):
    current_user_id = user_id
    user_movies = movies_with_tags_with_ratings[movies_with_tags_with_ratings.userId == current_user_id].title.unique()
    
    last_user_movie = user_movies[-1]
    
    movie_genres = title_genres[last_user_movie]
    
    movie_genres = change_string(movie_genres)

    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict)

    res = neigh.kneighbors(X_tfidf2, return_distance=True)
    
    movies_to_score = movies_with_tags_with_ratings.iloc[res[1][0]].sort_values(by = 'rating')[-10:].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [120]:
recommend_for_user(2)

Talented Mr. Ripley, The (1999) 5.0
Primal Fear (1996) 4.830852718306701
Pulp Fiction (1994) 4.817041336361281
Pulp Fiction (1994) 4.817041336361281
Funny Games U.S. (2007) 4.614990991887804
Dark Knight Rises, The (2012) 4.574637575182833
Silence (2016) 4.556576237363393
Dark Knight, The (2008) 4.54750733252021
Top Hat (1935) 4.457375446577545
13 Going on 30 (2004) 4.43183342372686
